# Introduction

The demand for detailed forest inventories is growing, fueled by the need for quality
data for harvest and afforestation planning, sustainable forest management and
conservation, modeling of carbon stock and carbon cycles, and more.
The increasing accessibility of UAVs and LiDAR sensors makes UAV LiDAR an ever more
valuable tool for conducting such inventories.
The technical characteristics of modern sensors alalow for very dense measurements,
which in turn introduce a shift from the area-based approach [@whiteABAGuide2013]
dominating early research and applications of LiDAR in forestry and still widely used in
the industry to an individual tree-based approach.
This shift brings the requirement for algorithms for automatic detection and
segmentation of individual trees within larger point clouds.
This problem can be solved well for forest stands that are either predominantly
coniferous or sparse, even by simple local maxima detection algorithms on rasterized
point clouds [@eysnAlpineITDBenchmark2015].
However, it remains an open challenge to develop a universal, robust detection approach
for dense mixed forests, in which the canopy structure is complex.
There is a growing body of research on developing and applying more complex and precise
methods, but most results that can be considered successful from the industrial adoption
point of view are in more mild forest types [@balsiSingletreeDetectionHighdensity2018,
@jeronimoApplyingLiDARIndividual2018].

In this study, we try to gain insight into the problem of tree detection and
segmentation by looking at its expected output. We analyze a collection of individual
trees manually identified in the large survey data over a dense mixed forest and explore
their features and properties.

In [1]:
#| label: imports

import glob

import laspy
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
#| label: prepare-data

file_paths = glob.glob("data/*/*.las")
file_paths.sort()

data = pd.DataFrame({"path": file_paths})
data["species"] = data["path"].map(lambda p: p.split("/")[-2])